## Setup (requires input or supervision)

### Packages, Gdrive mount, and logins (HuggingFace, WeightAndBiases)

In [ ]:
from google.colab import drive
import os
from getpass import getpass
import wandb
from huggingface_hub import login
import os
import json
import wandb
import datetime
import time
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, AutoModelForSequenceClassification

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TensorFlow info, warning, and error messages.

drive.mount('/content/drive', force_remount=True)

# Prompt for Hugging Face API token without saving it
# TODO remove api key
huggingface_token = None
if huggingface_token is None:
    huggingface_token = getpass("Please enter your Hugging Face API token: ")

# Set the token as an environment variable
os.environ["HUGGINGFACE_TOKEN"] = huggingface_token

# Log in using the token
login(token=os.environ["HUGGINGFACE_TOKEN"])

# Prompt for Weights & Biases (wandb) login
# TODO remove api key
wandb_key = None
if wandb_key is None:
    wandb_key = getpass("Please enter your Weights & Biases (wandb) API key: ")

wandb.login(key=wandb_key)

Mounted at /content/drive
Please enter your Hugging Face API token: ··········
Please enter your Weights & Biases (wandb) API key: ··········


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Experiment set up with config

In [ ]:
# TODO: the config will be imported from github repo later
# TODO: add llama3b setup
experiment_id = "entropy_illegal" + datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
experiment_desc = "eval only on entropy50k, 10 evals on mmlu mix, 10 MNT"

experiment_config = {
    "experiment_id": experiment_id,
    "experiment_desc": experiment_desc,
    "dataset_base_dir": "/content/drive/MyDrive/active-llm/datasets",
    "hf_model_name": "active-llm-winner-"+experiment_id,
    "model_name": "meta-llama/Llama-3.2-1B",
    "repo_id_base": "coderGenMC/active-llm-datasets", # dataset
    "hf_repo_name": "rnjs1992",
    "dataset_name": "rnjs1992/active-llm-datasets_acquired_dataset_entropy_budget50000_iter2", # for finetuning
    "hf_eval_dataset_name": "cais/mmlu", # some specs in eval code are hard coded, so cannot change this for now
    "eval_max_length": 2048,
    "num_train_epochs": 3,
    "eval_sample_size": 10,
    "max_new_tokens": 2048
}

# Write experiment configuration to a JSON file
config_file_path = "./experiment_config.json"
with open(config_file_path, "w") as config_file:
    json.dump(experiment_config, config_file, indent=4)

# Load configuration from the JSON file
with open(config_file_path, "r") as config_file:
    loaded_config = json.load(config_file)

print("Loaded Configuration:")
for key, value in loaded_config.items():
    print(f"{key}: {value}")

Loaded Configuration:
experiment_id: entropy_illegal20241214_011905
experiment_desc: eval only on entropy50k, 10 evals on mmlu mix, 10 MNT
dataset_base_dir: /content/drive/MyDrive/active-llm/datasets
hf_model_name: active-llm-winner-entropy_illegal20241214_011905
model_name: meta-llama/Llama-3.2-1B
repo_id_base: coderGenMC/active-llm-datasets
hf_repo_name: rnjs1992
dataset_name: rnjs1992/active-llm-datasets_acquired_dataset_entropy_budget50000_iter2
hf_eval_dataset_name: cais/mmlu
eval_max_length: 2048
num_train_epochs: 3
eval_sample_size: 10
max_new_tokens: 2048


### Clone Github repo (you need private-public keys with the same naming convention)

In [ ]:
# Create symbolic links to the SSH keys in Drive
!ln -s /content/drive/MyDrive/active-llm/ssh_keys/id_colab ~/.ssh/id_colab
!ln -s /content/drive/MyDrive/active-llm/ssh_keys/id_colab.pub ~/.ssh/id_colab.pub

# Start SSH agent and add key
!eval "$(ssh-agent -s)"
!ssh-add ~/.ssh/id_colab

# Create SSH config
ssh_config = """
Host github.com
  HostName github.com
  User git
  IdentityFile ~/.ssh/id_colab
  StrictHostKeyChecking no
"""

# Create the .ssh directory if it doesn't exist
os.makedirs(os.path.expanduser("~/.ssh"), exist_ok=True)

with open(os.path.expanduser("~/.ssh/config"), "w") as f:
    f.write(ssh_config)

# Test SSH connection
!ssh -T git@github.com


Agent pid 3070
Could not open a connection to your authentication agent.
Hi kwonosub! You've successfully authenticated, but GitHub does not provide shell access.


In [ ]:
repo_ssh_url = "git@github.com:masoudcharkhabi/ML-from-Expert-Preferences.git"
branch_name = "abstractions"
!git clone -b {branch_name} --single-branch {repo_ssh_url}
repo_name = "ML-from-Expert-Preferences"
os.chdir(repo_name)
!git branch
!ls -ltr

Cloning into 'ML-from-Expert-Preferences'...
remote: Enumerating objects: 365, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 365 (delta 61), reused 99 (delta 43), pack-reused 233 (from 1)
Receiving objects: 100% (365/365), 462.44 KiB | 1.86 MiB/s, done.
Resolving deltas: 100% (178/178), done.
* abstractions
total 304
-rw-r--r-- 1 root root   3152 Dec 14 01:19 README.md
drwxr-xr-x 3 root root   4096 Dec 14 01:19 baseline
-rw-r--r-- 1 root root 261118 Dec 14 01:19 colab_experiments.ipynb
drwxr-xr-x 4 root root   4096 Dec 14 01:19 cs329h-project
-rw-r--r-- 1 root root   1612 Dec 14 01:19 serve.py
-rw-r--r-- 1 root root    190 Dec 14 01:19 requirements.txt
-rw-r--r-- 1 root root  10694 Dec 14 01:19 eval.py
-rw-r--r-- 1 root root   5002 Dec 14 01:19 data_prep.py
drwxr-xr-x 5 root root   4096 Dec 14 01:19 data
-rw-r--r-- 1 root root   3864 Dec 14 01:19 train.py


In [ ]:
!pip install -r requirements.txt

#TODO figure out the verions and added to requirements or .yml later
!pip install datasets
!pip install torch
!pip install evaluate
!pip install rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=872e8e5adbcd85cb26ce962117ad2a75262f874f2b172a36217db5a355ecfdd4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's d

### Make sure you have a GPU and High memory

In [ ]:
# Check for gpu
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Check for high ram
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('\nYour runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Sat Dec 14 01:19:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              49W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Data prep

### data_prep.py (this will be imported from the repo later)

In [ ]:
# data_prep.py
from datasets import load_dataset
import pyarrow as pa
import pyarrow.parquet as pq
from datasets import Dataset, concatenate_datasets
from huggingface_hub import HfApi, HfFolder, create_repo

class DatasetConverterUploader:
    def __init__(self, dataset_base_dir, repo_id_base):
        self.dataset_base_dir = dataset_base_dir
        self.repo_id_base = repo_id_base
        self.dataset_directories = [
            os.path.join(dataset_base_dir, d) for d in os.listdir(dataset_base_dir) if os.path.isdir(os.path.join(dataset_base_dir, d))
        ]
        self.token = HfFolder.get_token()
        self.api = HfApi()

    def convert_arrow_to_parquet(self):
        for dataset_dir in self.dataset_directories:
            parquet_file_path = os.path.join(dataset_dir, 'data.parquet')
            if os.path.exists(parquet_file_path):
                print(f"Parquet file already exists in {dataset_dir}, skipping conversion...")
                continue

            # Handle multiple Arrow files
            arrow_files = [
                os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('.arrow')
            ]
            if not arrow_files:
                print(f"No Arrow files found in {dataset_dir}, skipping...")
                continue

            # Load all Arrow files into a single Hugging Face Dataset object
            datasets = [Dataset.from_file(arrow_file) for arrow_file in arrow_files]
            dataset = concatenate_datasets(datasets)

            # Convert to Apache Arrow Table
            table = pa.concat_tables([ds.data.table for ds in datasets])

            # Save as Parquet file
            pq.write_table(table, parquet_file_path)

            print(f"Converted Arrow files in {dataset_dir} to {parquet_file_path}")

    def upload_parquet_to_hub(self):
        for dataset_dir in self.dataset_directories:
            parquet_file_path = os.path.join(dataset_dir, 'data.parquet')

            if not os.path.exists(parquet_file_path):
                print(f"Parquet file not found in {dataset_dir}, skipping...")
                continue

            # Create a unique repository ID for each dataset
            dataset_name = os.path.basename(dataset_dir)
            repo_id = f"{self.repo_id_base}_{dataset_name}"

            # Create the repository if it does not exist
            try:
                create_repo(repo_id, repo_type="dataset", token=self.token, exist_ok=True)
            except Exception as e:
                print(f"Error creating repository {repo_id}: {e}")
                continue

            # Upload the file
            self.api.upload_file(
                path_or_fileobj=parquet_file_path,
                path_in_repo=os.path.basename(parquet_file_path),
                repo_id=repo_id,
                repo_type="dataset",
                token=self.token
            )

            print(f"Uploaded {parquet_file_path} to Hugging Face Hub with repo ID {repo_id}")

class DataPreparation:
    def __init__(self, dataset_path: str):
        self.dataset_path = dataset_path
        self.dataset = None

    def load_data(self):
        """Load dataset from Hugging Face"""
        self.dataset = load_dataset(self.dataset_path)
        return self.dataset

    def preprocess(self, example):
        """Preprocess dataset into input-output pairs"""
        return {
            "input_text": example['inputs'],
            "target_text": example['targets'],
        }

    def tokenize_function(self, examples, tokenizer):
        """Tokenize input and output text"""
        model_inputs = tokenizer(
            examples["input_text"],
            max_length=2048,
            truncation=True,
            padding="max_length",
        )
        labels = tokenizer(
            examples["target_text"],
            max_length=2048,
            truncation=True,
            padding="max_length",
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    def prepare_dataset(self, tokenizer):
        """Prepare the dataset for training"""
        train_dataset = self.dataset["train"].map(self.preprocess)
        train_dataset = train_dataset.map(lambda x: self.tokenize_function(x, tokenizer), batched=True)
        eval_dataset = self.dataset["validation"].map(self.preprocess) if "validation" in self.dataset else None
        eval_dataset = eval_dataset.map(lambda x: self.tokenize_function(x, tokenizer), batched=True) if eval_dataset else None
        # Return only train_dataset instead of a tuple
        return train_dataset

    def dataset_info(self):
        """Print information about the dataset, such as the size"""
        if self.dataset:
            print(f"Dataset name: {self.dataset_path}")
            for split in self.dataset.keys():
                print(f"Split: {split}, Number of examples: {len(self.dataset[split])}")
        else:
            print("Dataset is not loaded. Please call load_data() first.")

### Load parquet datasets to HF

In [ ]:
dataset_base_dir = loaded_config['dataset_base_dir']
repo_id_base = loaded_config['repo_id_base']
converter_uploader = DatasetConverterUploader(dataset_base_dir, repo_id_base)
converter_uploader.convert_arrow_to_parquet()
converter_uploader.upload_parquet_to_hub()

Parquet file already exists in /content/drive/MyDrive/active-llm/datasets/acquired_dataset_entropy_budget25000, skipping conversion...
Parquet file already exists in /content/drive/MyDrive/active-llm/datasets/acquired_dataset_entropy_budget50000, skipping conversion...
Parquet file already exists in /content/drive/MyDrive/active-llm/datasets/acquired_dataset_random_budget25000, skipping conversion...
Parquet file already exists in /content/drive/MyDrive/active-llm/datasets/acquired_dataset_confidence_budget50000, skipping conversion...
Parquet file already exists in /content/drive/MyDrive/active-llm/datasets/acquired_dataset_random_budget50000, skipping conversion...
Parquet file already exists in /content/drive/MyDrive/active-llm/datasets/acquired_dataset_confidence_budget25000, skipping conversion...
Parquet file already exists in /content/drive/MyDrive/active-llm/datasets/acquired_dataset_entropy_budget25000_iter2, skipping conversion...
Parquet file already exists in /content/drive

KeyboardInterrupt: 

### Data prep usage

In [ ]:
from re import M
from transformers import AutoTokenizer
# from data_prep import DataPreparation

# Load tokenizer
model_name = loaded_config['model_name'] #
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add the padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token}) # tokenizer.eos_token is used as padding token for Llama models

# Create instance of DataPreparation
dataset_name = loaded_config['dataset_name'] #
data_preparation = DataPreparation(dataset_path=dataset_name)

# Load the dataset
dataset = data_preparation.load_data()

# Prepare the dataset (tokenize)
train_dataset = data_preparation.prepare_dataset(tokenizer=tokenizer)

# Now train_dataset is ready for training
# Get dataset information
data_preparation.dataset_info()

# Get experiment_id
print(f"experiment_id: {loaded_config['experiment_id']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

data.parquet:   0%|          | 0.00/163M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset name: rnjs1992/active-llm-datasets_acquired_dataset_entropy_budget50000_iter2
Split: train, Number of examples: 50000
experiment_id: entropy_illegal20241214_011905


# Fine-tune

### train.py (This will be imported from GH later and requires a Weights and Biases (wandb) API key for logging)

In [ ]:
# train.py
import os
import datetime
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import wandb

class ModelTrainer:
    def __init__(self, model_name: str, experiment_id: str, loaded_config):
        self.model_name = model_name
        self.experiment_id = experiment_id
        self.loaded_config = loaded_config
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            offload_folder="./offload"  # Folder to store offloaded parts of the model
        )
        self.trainer = None
        self.output_dir = f"./models/fine_tuned_model_{self.experiment_id}"
        # Initialize WandB with config
        wandb.init(
            project="active-llm",
            name=f"fine_tune_{self.experiment_id}",
            config=loaded_config,  # Add experiment config as metadata
            resume="allow"
        )

    def setup_training(self, train_dataset, eval_dataset=None, tokenizer=None):
        """Set up training arguments and Trainer"""

        # Ensure the output directory exists
        os.makedirs(self.output_dir, exist_ok=True)

        # Adjust the tokenizer to a reduced max length to reduce memory

        # Training arguments without DeepSpeed and offloading
        training_args = TrainingArguments(
            output_dir=self.output_dir,
            # evaluation_strategy="epoch" if eval_dataset is not None else "no",
            evaluation_strategy="no",
            learning_rate=2e-5,
            per_device_train_batch_size=2,  # Reduce batch size to avoid memory issues
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=4,  # Reduce gradient accumulation to lower memory needs
            optim="adamw_torch",
            lr_scheduler_type="linear",
            warmup_ratio=0.03,
            num_train_epochs=self.loaded_config['num_train_epochs'],
            weight_decay=0,
            report_to=["wandb"],
            run_name="model_training",
            fp16=True,
            gradient_checkpointing=True,
            seed=42,
            logging_steps=250,      # Log metrics to wandb every n steps
            save_strategy="steps",
            save_steps=3125,      # Save a checkpoint every m steps
        )

        # Use DataCollatorForLanguageModeling for data handling
        data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            # eval_dataset=eval_dataset,
            data_collator=data_collator
        )

    def train_model(self, save_model: bool = True):
        """Train the model"""
        if self.trainer is not None:
            try:
                self.trainer.train()
                if save_model:
                    # Ensure the output directory exists before saving
                    os.makedirs(self.output_dir, exist_ok=True)
                    self.trainer.save_model(self.output_dir)
                    # Manually add model_type to config
                    self.model.config.model_type = "llama"
                    self.model.config.save_pretrained(self.output_dir)
                    print(f"Model saved to: {self.output_dir}")
            except RuntimeError as e:
                print("RuntimeError occurred:", e)
            finally:
                wandb.finish()
        else:
            raise ValueError("Trainer is not set up. Please call setup_training() first.")


### Fine-tune usage

In [ ]:
print(f"Allocated memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Max memory: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")
print(f"Total memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Set CUDA_LAUNCH_BLOCKING to synchronize CUDA operations for better debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Set PYTORCH_CUDA_ALLOC_CONF for better memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear GPU memory cache
torch.cuda.empty_cache()

Allocated memory: 0.00 GB
Max memory: 0.00 GB
Total memory: 42.48 GB


In [ ]:
# from train import ModelTrainer

trainer = ModelTrainer(model_name=model_name,
                       experiment_id=experiment_id,
                       loaded_config=loaded_config)
trainer.setup_training(train_dataset=train_dataset,
                       tokenizer=tokenizer)
trainer.train_model()

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

wandb: Currently logged in as: kwonosub-ai (ai-eval). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
250,1.952600
500,1.733500
750,1.710000
1000,1.633200
1250,1.634900
1500,1.633300
1750,1.593200
2000,1.580500
2250,1.561600
2500,1.575800


Model saved to: ./models/fine_tuned_model_entropy_illegal20241214_011905


train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
train/grad_norm,█▄▇▃█▆▃▅▃▄▁▄▄▅▄▅▃▃▂▃▅▆▄▇▃▄▄▃▂▄▁▃▄▄▃▃▃▁▄▅
train/learning_rate,▄████▇▇▇▇▇▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▇▇▆▆▆▆▆▆▆▅▅▅▅▅▃▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,1.7937029136384e+18
train/epoch,3
train/global_step,18750
train/grad_norm,4.7474
train/learning_rate,0.0
train/loss,0.647


### Load the finetuned model to HF

In [ ]:
# Load to HF
# Authenticate with Hugging Face
login(token=huggingface_token)

# Define paths and names
fine_tuned_model_name = "fine_tuned_model_"+experiment_id

hf_model_name = loaded_config['hf_model_name']
hf_repo_name = loaded_config['hf_repo_name']
repo_id = hf_repo_name + "/" + hf_model_name

model = AutoModelForSequenceClassification.from_pretrained("./models/" + fine_tuned_model_name)
# model = AutoModelForCausalLM.from_pretrained("./models/" + fine_tuned_model_name)

# Create README content with model name
readme_content = f"""
    ----------------------------------------------------------------------------
    language: en
    tags:
    - sequence-classification
    - transformers
    - t5
    license: apache-2.0
    ---

    # {hf_model_name}

    **Model Name:** `{hf_model_name}`
    **Model Owner:** [coderGenMC](https://huggingface.co/coderGenMC)

    ## Model Description

    This model is a fine-tuned version of T5 for sequence classification tasks. It has been trained to classify text inputs into predefined categories.

    ## Training Details

    - **Model Architecture:** T5
    - **Fine-Tuned For:** Sequence Classification

    ## How to Use

    You can use this model directly with the Transformers library

    ----------------------------------------------------------------------------
    """

# Create the directory if it doesn't exist
save_directory = "./models/" + fine_tuned_model_name
os.makedirs(save_directory, exist_ok=True)

# Define the path for README.md
readme_path = os.path.join(save_directory, "README.md")

# Write the readme_content to README.md
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(readme_content)

model.save_pretrained(hf_model_name)
tokenizer.push_to_hub(repo_id)
model.push_to_hub(repo_id, use_auth_token=huggingface_token)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at ./models/fine_tuned_model_entropy_illegal20241214_011905 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rnjs1992/active-llm-winner-entropy_illegal20241214_011905/commit/6af1b82479f8ad8a894aaccfdd965187b901f4a2', commit_message='Upload LlamaForSequenceClassification', commit_description='', oid='6af1b82479f8ad8a894aaccfdd965187b901f4a2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rnjs1992/active-llm-winner-entropy_illegal20241214_011905', endpoint='https://huggingface.co', repo_type='model', repo_id='rnjs1992/active-llm-winner-entropy_illegal20241214_011905'), pr_revision=None, pr_num=None)

# Eval

In [ ]:
import random
import numpy as np
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import logging as transformers_logging
import torch
import wandb
from tqdm import tqdm


eval_model = 'rnjs1992/active-llm-winner-mean_margin5000020241213_101351'
eval_sample_size = 1000

model = AutoModelForCausalLM.from_pretrained(eval_model)

tokenizer = AutoTokenizer.from_pretrained(eval_model)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load the MMLU dataset from Hugging Face with the specified subject configuration
mmlu_dataset = load_dataset("cais/mmlu", "all")
test_dataset = mmlu_dataset["test"]
dev_dataset = mmlu_dataset["dev"]


random.seed(54)
random_index = random.sample(range(len(test_dataset)), eval_sample_size)
print(random_index)
test_dataset = test_dataset.select(random_index)


config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

[2252, 7191, 9110, 4881, 7913, 7987, 6232, 3588, 7315, 5407, 9459, 11981, 6260, 3420, 11857, 3277, 917, 8120, 13101, 8993, 7419, 9792, 1490, 8882, 5572, 8450, 4804, 6796, 4475, 8912, 2333, 1585, 10576, 4278, 4555, 7172, 9611, 607, 6160, 11095, 207, 5933, 4354, 6, 3756, 11926, 337, 2786, 12019, 2235, 6208, 3977, 424, 11476, 3408, 7321, 6233, 9488, 2897, 7195, 3917, 4006, 10350, 9780, 262, 10598, 7946, 9125, 3854, 13323, 5570, 6444, 6869, 6997, 6361, 5972, 9031, 310, 13912, 13718, 7766, 13808, 7553, 4832, 3537, 5812, 7929, 12551, 4707, 1330, 959, 10748, 8285, 7334, 3766, 8262, 7586, 2622, 12737, 8132, 9687, 5673, 8227, 4993, 5226, 4489, 12611, 12815, 13060, 12949, 722, 1593, 9906, 5233, 6295, 6567, 6899, 6552, 2177, 7043, 5994, 5194, 7945, 7148, 12218, 9466, 12906, 1602, 2307, 8789, 9138, 5562, 7563, 13204, 13722, 4114, 9753, 10781, 2163, 6879, 5621, 4569, 11639, 13141, 2063, 12664, 8287, 9123, 11522, 13663, 13416, 4810, 11454, 4711, 1732, 7636, 12956, 1548, 3298, 9706, 2014, 2496, 2914,

In [ ]:
def format_example(dataset, index, include_answer=True):
    # derive question for this example
    prompt = dataset[index]['question']

    choice_indices = ['A', 'B', 'C', 'D']
    choice_contents = dataset[index]['choices']

    for j, choice in enumerate(choice_indices):
      prompt += "\n{}. {}".format(choice, choice_contents[j])
    prompt += "\nAnswer:"

    if include_answer:
        prompt += " {}\n\n".format(dataset[index]['answer'])
    return prompt

def gen_prompt(dev_dataset, subject):
    prompt = "The following are multiple choice questions (with answers) about {}.\n\n".format(format_subject(subject))
    k = len(dev_dataset)
    for i in range(k):
        prompt += format_example(dev_dataset, i)
    return prompt

def format_subject(subject):
    l = subject.split("_")
    s = ""
    for entry in l:
        s += " " + entry
    return s

In [ ]:
cors = []

answers = ['A', 'B', 'C', 'D']
answer_tokens = [tokenizer.convert_tokens_to_ids(ans) for ans in answers]

for i in tqdm(range(len(test_dataset))):
    # get example, its subject, and its answer
    example = test_dataset[i]
    example_subject = example['subject']

    label = example['answer']

    # get dev set
    example_dev_dataset = dev_dataset.filter(lambda example: example["subject"]==example_subject)

    # make prompt
    train_prompt = gen_prompt(example_dev_dataset, example_subject)
    question_prompt = format_example(test_dataset, i, include_answer=False)
    prompt = train_prompt + question_prompt

    # get logits given the prompt
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[:, -1, answer_tokens]
        pred = logits.argmax(dim=-1)
        cor = pred == label
        cors.append(cor)
print(np.mean(cors))


100%|██████████| 1000/1000 [56:08<00:00,  3.37s/it]

0.27
